https://www.ctcms.nist.gov/fipy/documentation/glossary.html#term-trilinos

In [ ]:
# Complicated way to import finis if not installed

import os
import sys
finis_path = "../tp" #Folder containing finis folder
finis_abs_path = os.path.abspath(finis_path)
sys.path.append(finis_abs_path)

import finis
import numpy as np
import scipy.sparse as sp
import scipy
import pyamg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

In [ ]:
%matplotlib notebook

### Setup

In [ ]:
mesh = finis.triangulate(max_area=0.002)
fe_u = finis.fe_space(mesh, order=2, order_int=2)
fe_p = finis.fe_space(mesh, order=1, order_int=2)
finis.plot_mesh(mesh)

assert np.array_equal(fe_u['integ'], fe_p['integ']), "FE Spaces not compatible"
assert np.array_equal(fe_u['w'], fe_p['w']), "FE integration weights not compatible"

In [ ]:
f1 = lambda x,y: np.zeros_like(x)
# f2 = lambda x,y: -np.ones_like(y)
f2 = lambda x,y: np.zeros_like(x)
up1 = lambda x,y: np.zeros_like(x)
up2 = lambda x,y: x*(1-x)

dim_u1 = fe_u['U'].shape[1]
dim_u2 = dim_u1
dim_p = fe_p['U'].shape[1]
dim_w = fe_u['w'].size

W = sp.spdiags(fe_u['w'], [0], m=dim_w, n=dim_w)


# LHS
A11 = fe_u['DUX'].transpose().dot(W).dot(fe_u['DUX']) + fe_u['DUY'].transpose().dot(W).dot(fe_u['DUY'])
A22 = A11
A13 = -fe_u['DUX'].transpose().dot(W).dot(fe_p['U'])
A23 = -fe_u['DUY'].transpose().dot(W).dot(fe_p['U'])

# _row = U.transpose().dot(W).dot(np.ones((dim_w,)))[None, :] # Force mean(p) = 0
A = sp.bmat([[A11, None, A13], [None, A22, A23], [A13.transpose(), A23.transpose(), None]], format='csr')
assert np.allclose((A - A.transpose()).data, 0), "A is not numerically symmetric!"

# RHS
F1int = f1(fe_u['integ'][:,0], fe_u['integ'][:,1])
F2int = f2(fe_u['integ'][:,0], fe_u['integ'][:,1])
UP1 = up1(fe_u['dof'][:,0], fe_u['dof'][:,1])
UP2 = up2(fe_u['dof'][:,0], fe_u['dof'][:,1])

_F1 = np.concatenate((
    fe_u['U'].transpose().dot(W).dot(F1int),
    fe_u['U'].transpose().dot(W).dot(F2int),
    np.zeros(dim_p, )
))
_F2 = np.concatenate((
    A11.dot(UP1),
    A22.dot(UP2),
    np.zeros(dim_p, )
))
F = _F1 - _F2

### Dirichlet

In [ ]:
row = np.where(fe_u['markers']==0)[0]
col = np.arange(row.size)
data = np.ones((row.size, ), dtype=np.float)
P = sp.csr_matrix((data, (row, col)), shape=(fe_u['markers'].size, row.size))

dim_u1d = P.shape[1]
dim_u2d = P.shape[1]

A11d = P.transpose().dot(A11).dot(P)
A22d = P.transpose().dot(A22).dot(P)
A13d = P.transpose().dot(A13)
A23d = P.transpose().dot(A23)

Ad = sp.bmat([[A11d, None, A13d], [None, A22d, A23d], [A13d.transpose(), A23d.transpose(), None]], format='csr')
assert np.allclose((Ad - Ad.transpose()).data, 0), "A is not numerically symmetric!"

_F1d = np.concatenate((
    P.transpose().dot(fe_u['U'].transpose()).dot(W).dot(F1int),
    P.transpose().dot(fe_u['U'].transpose()).dot(W).dot(F2int),
    np.zeros(dim_p, )
))
_F2d = np.concatenate((
    P.transpose().dot(A11).dot(UP1),
    P.transpose().dot(A22).dot(UP2),
    np.zeros(dim_p, )
))
Fd = _F1d - _F2d

In [ ]:
if A.shape[0] <= 100:
    max_val = max([
        abs(A.min()),
        abs(A.max()),
        np.amax(np.abs(F))
    ])

    f, (a0, a1) = plt.subplots(1,2, gridspec_kw = {'width_ratios':[A.shape[0], 1]}, sharey=True, figsize=(5,4))
    a0.pcolor(A.toarray(), vmin=-max_val, vmax=max_val)
    a0.invert_yaxis()
    a0.axis("equal")
    a0.set_title("A")

    mappable = a1.pcolor(F[:, None], vmin=-max_val, vmax=max_val)
    a1.axis("equal")
    a1.set_title("F")

    plt.suptitle("Sparsity Patterns")
    plt.show()
else:
    print("A  too large for plotting ({})".format(A.shape[0]))
    
if Ad.shape[0] <= 300:
    max_val = max([
        abs(Ad.min()),
        abs(Ad.max()),
        np.amax(np.abs(Fd))
    ])

    f, (a0, a1) = plt.subplots(1,2, gridspec_kw = {'width_ratios':[Ad.shape[0], 1]}, sharey=True, figsize=(5,4))
    a0.pcolor(Ad.toarray(), vmin=-max_val, vmax=max_val)
    a0.invert_yaxis()
    a0.axis("equal")
    a0.set_title("A")

    mappable = a1.pcolor(Fd[:, None], vmin=-max_val, vmax=max_val)
    a1.axis("equal")
    a1.set_title("F")

    plt.suptitle("Sparsity Patterns: Dirichlet")
    plt.show()
else:
    print("Ad too large for plotting ({})".format(Ad.shape[0]))
    
if Ad.shape[0] <= 500:
    null = scipy.linalg.null_space(Ad.toarray())
    plt.figure()
    plt.plot(null)
    plt.title("Null Space (dim = {})".format(null.shape[1]))
    plt.show()
else:
    print("Ad too large for null space analysis")

In [ ]:
%%time
tol=1e-8
residuals=[]
x, ml = pyamg.solve(Ad, Fd, verb=True, tol=tol, residuals=residuals, return_solver=True)
x[2*dim_u1d:] -= np.mean(x[2*dim_u1d:]) # remove mean pressure
print("\n--- ML Solver ---")
print(ml)

print("--- Convergence ---")
print("Geometric factor {}".format((residuals[-1]/residuals[0])**(1.0/len(residuals))))

print("\n--- Time ---")

In [ ]:
%%time
x_sp, info = sp.linalg.cg(Ad, Fd, tol=tol)
x_sp[2*dim_u1d:] -= np.mean(x_sp[2*dim_u1d:]) # remove mean pressure

flag_dict = {
    0: "successful exit",
    -1: "illegal input or breakdown",
    +1: "convergence to tolerance not achieved or number of iterations <0"
}
print("--- SciPy CG ---")
rel_error = np.amax(np.abs(x-x_sp))/np.amax(np.abs(x_sp))
print("Relative Error Inf Norm: {}".format(rel_error))
if rel_error > 100*tol:
    print("Error too big, using SciPy solution")
    x = x_sp

print("\n--- Convergence ---")
print("Flag: {} -> {}".format(info, flag_dict[np.sign(info)]))

print("\n--- Time ---")


In [ ]:
u1d_h = P.dot(x[0:dim_u1d])
u2d_h = P.dot(x[dim_u1d:2*dim_u1d])
p_h = x[2*dim_u1d:]
assert p_h.size == dim_p

fig = plt.figure(figsize=(9,5))
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.plot_trisurf(fe_u['dof'][:,0], fe_u['dof'][:,1], u1d_h + UP1, linewidth=0.2, antialiased=True)
ax.set_title("U_x")
ax.set_xlabel("x")
ax.set_ylabel("y")

ax = fig.add_subplot(1, 3, 2, projection='3d', sharez=ax)
ax.plot_trisurf(fe_u['dof'][:,0], fe_u['dof'][:,1], u2d_h + UP2, linewidth=0.2, antialiased=True)
ax.set_title("U_y")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.plot_trisurf(fe_p['dof'][:,0], fe_p['dof'][:,1], p_h, linewidth=0.2, antialiased=True)
ax.set_title("P")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

print("Max pressure diff {} (should be 2)".format(np.amax(p_h)-np.amin(p_h)))

In [ ]:
shading= 'flat' # flat or gouraud
max_u = max([
    np.amax(u1d_h + UP1),
    np.amax(u2d_h + UP2),
])
min_u = max([
    np.amin(u1d_h + UP1),
    np.amin(u2d_h + UP2),
])

fig = plt.figure(figsize=(9,5))
ax = fig.add_subplot(1, 3, 1)
plt.tripcolor(fe_u['dof'][:,0], fe_u['dof'][:,1], u1d_h + UP1, shading=shading, vmin=min_u, vmax=max_u)
ax.set_title("U_x")
ax.set_xlabel("x")
plt.colorbar()

fig.add_subplot(1, 3, 2, sharex=ax, sharey=ax)
plt.tripcolor(fe_u['dof'][:,0], fe_u['dof'][:,1], u2d_h + UP2, shading=shading, vmin=min_u, vmax=max_u)
plt.title("U_y")
plt.xlabel("x")
plt.show()

ax = fig.add_subplot(1, 3, 3, sharex=ax, sharey=ax)
plt.tripcolor(fe_p['dof'][:,0], fe_p['dof'][:,1], p_h,  shading=shading)
plt.title("P")
plt.xlabel("x")
plt.colorbar()

plt.tight_layout()
plt.show()
print("Max pressure diff {} (should be 2)".format(np.amax(p_h)-np.amin(p_h)))